In [1]:
from learner import Trainer, EvaluateOnTest
from model import SpanEmo
from data_loader import DataClass
from data_selector import DataSelector
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score
import json

seed = 12345678

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if str(device) == 'cuda:0':
    print("Currently using GPU: {}".format(device))
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    print("WARNING: USING CPU")

Currently using GPU: cuda:0


In [3]:
def make_loaders(args, data, batch_size, shuffle=True):
    dataset = DataClass(args, data)
    data_loader = DataLoader(dataset,
                             batch_size=int(batch_size),
                             shuffle=shuffle)

    print('The number of batches: ', len(data_loader))

    return data_loader

In [4]:
def make_model(args):
    model = SpanEmo(output_dropout=args['output_dropout'],
                    backbone=args['backbone'])
    
    return model


In [5]:
def pipeline(args, loaders=None):
    now = datetime.datetime.now()
    filename = now.strftime("%Y-%m-%d-%H:%M:%S")
    fw = open('configs/' + filename + '.json', 'a')
    json.dump(args, fw, sort_keys=True, indent=2)

    train_data_loader, val_data_loader = loaders
    model = make_model(args)

    learn = Trainer(model, train_data_loader, val_data_loader, filename=filename)
    learn.fit(
        num_epochs=int(args['max_epoch']),
        args=args,
        device=device
    )

In [6]:
hyperparams = {
    'train_path':'data/train.csv', 
    'val_path':'data/val.csv',
    'backbone':'bert-base-uncased',
    'train_batch_size': 32,
    'val_batch_size': 32,
    'output_dropout': 0.1,
    'max_epoch': 20,
    'max_length': 512,
    'ffn_lr': 0.001,
    'bert_lr': 2e-5
}

In [7]:
assert False

AssertionError: 

## Train

In [8]:
ds = DataSelector("data.csv")
train, val, tests = ds.select_data(ratio={"Cornell": 1})

Filtering...
Calculating counts...
Grabbing data...


In [ ]:
for i in train['text'].values:
    s = sum([len(x.split()) for x in i])
    if s > 256:
        print(s)
        # print(i)

282
266
282
281


In [ ]:
m = 0
c = 0
for i in train_loader:
    # print(len(i[2]))
    ma = max(i[0]['attention_mask'].sum(axis=1))
    if ma > 256:
        # print(i[0]['input_ids'])
        print(ma)
        m = ma
        c += 1

print(m)
print(c)

tensor(303)
tensor(269)
tensor(271)
tensor(284)
tensor(257)
tensor(296)
tensor(260)
tensor(334)
tensor(292)
tensor(275)
tensor(303)
tensor(293)
tensor(295)
tensor(275)
tensor(324)
tensor(371)
tensor(285)
tensor(275)
tensor(257)
tensor(357)
tensor(260)
tensor(309)
tensor(265)
tensor(298)
tensor(271)
tensor(330)
tensor(285)
tensor(272)
tensor(329)
tensor(295)
tensor(290)
tensor(282)
tensor(300)
tensor(306)
tensor(262)
tensor(263)
tensor(265)
tensor(293)
tensor(334)
tensor(265)
tensor(269)
tensor(310)
tensor(394)
tensor(276)
tensor(282)
tensor(330)
tensor(360)
tensor(288)
tensor(324)
tensor(266)
tensor(262)
tensor(344)
tensor(291)
tensor(281)
tensor(257)
tensor(268)
tensor(323)
tensor(277)
tensor(300)
tensor(262)
tensor(387)
tensor(372)
tensor(267)
tensor(276)
tensor(305)
tensor(352)
tensor(335)
tensor(323)
tensor(261)
tensor(258)
tensor(316)
tensor(293)
tensor(258)
tensor(309)
tensor(399)
tensor(303)
tensor(262)
tensor(275)
tensor(273)
tensor(275)
tensor(358)
tensor(276)
tensor(286)
tens

In [ ]:
train_loader = make_loaders(hyperparams, train, hyperparams['train_batch_size'])
val_loader = make_loaders(hyperparams, val, hyperparams['val_batch_size'])
loaders = (train_loader, val_loader)

/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter_2018 - 1grams ...


PreProcessing dataset ...: 100%|██████████| 52668/52668 [02:33<00:00, 342.89it/s]


The number of batches:  1646
Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...


PreProcessing dataset ...: 100%|██████████| 6583/6583 [00:19<00:00, 331.67it/s]


The number of batches:  206


In [ ]:
#wooooooooooooooooooooo
pipeline(hyperparams, loaders=loaders)

## Test

In [9]:
def test(args, model_path, loader=None):

    if loader is None:
        test_data_loader = make_loaders(args, test=True)
    else:
        test_data_loader = loader

    model = make_model(args)
    
    learn = EvaluateOnTest(model, test_data_loader, model_path='models/' + model_path)
    return learn.predict(device=device), model

In [10]:
hyperparams['test_path'] = 'data/test.csv'
hyperparams['test_batch_size'] = 32

In [11]:
test_loader = make_loaders(hyperparams, tests, hyperparams['test_batch_size'], shuffle=False)

/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter_2018 - 1grams ...


PreProcessing dataset ...: 100%|██████████| 6584/6584 [00:18<00:00, 346.62it/s]

The number of batches:  206


In [12]:
preds, model = test(hyperparams, "2023-11-25-15:31:26_checkpoint.pt", loader=test_loader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RMSE-Macro: 0.1302 RMSE-Micro: 0.1066 MSE-Micro: 0.0170 Time: 00:43


In [63]:
dc = DataClass(hyperparams, [
    [
        "I'M A GENIUS. IM SO HAPPY", 
        "i'm so sad. nothing is working", 
        "i'm super excited", 
        "i'm so depressed"
    ], [
        "I just got a promotion at work! I'm so excited and proud of myself!",
        "Wow, that's fantastic news! Congratulations! 🎉",
        "Thanks! It's been a long journey, but I finally feel recognized for my efforts.",
        "I can imagine the hard work you've put in. Your dedication has paid off. How do you feel now?",
    ]], pred_mode=True)
dl = DataLoader(dc, shuffle=False)

Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...


PreProcessing dataset ...: 100%|██████████| 1/1 [00:00<00:00, 329.61it/s]


In [64]:
logits = model.predict(next(iter(dl)), device)[2].cpu().detach().numpy()
probs = 1/(1+np.exp(-logits))[0]


In [65]:
label_names = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", "joy", "love", "nervous", "optimism", "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"]

for i in range(28):
    print(f"{probs[i]:.3f}", label_names[i])


0.090 admiration
0.022 amusement
0.037 anger
0.046 annoyance
0.121 approval
0.039 caring
0.052 confusion
0.117 curiosity
0.026 desire
0.050 disappointment
0.081 disapproval
0.009 disgust
0.003 embarrassment
0.073 excitement
0.018 fear
0.008 gratitude
0.004 grief
0.060 joy
0.039 love
0.007 nervous
0.074 optimism
0.002 pride
0.026 realization
0.003 relief
0.017 remorse
0.063 sadness
0.051 surprise
0.537 neutral


In [ ]:
label_names = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", "joy", "love", "nervous", "optimism", "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"]

y_true = pd.DataFrame(preds['y_true'], columns=label_names)
y_pred = pd.DataFrame(preds['y_pred'], columns=label_names)
logits = pd.DataFrame(preds['logits'], columns=label_names)

correlation = "pearson"

y_true_corr = y_true.corr(correlation)
y_pred_corr = y_pred.corr(correlation)
logits_corr = logits.corr(correlation)

In [ ]:
# _ = sns.clustermap(logits_corr, annot=False)

In [ ]:
# data = pd.read_csv("data/test.csv")

# wrongs = [(i[0], i[1].sum(), [(j, label_names[j]) for j, x in enumerate(i[1]) if x]) for i in enumerate(preds['y_true'] != preds['y_pred']) if i[1].any()]
# test_data = [(i, data['text'][i], [(j, label_names[j]) for j in range(28) if data[str(j)][i]]) for i in data.index]
# test_data = [test_data[i[0]] for i in wrongs]